# SLU08 - Classification With Logistic Regression: Exercise notebook

In [1]:
import pandas as pd 
import numpy as np 
import hashlib

In this notebook you will practice the following: 

    - What classification is for
    - Logistic regression
    - Cost function
    - Binary classification
    
You thought that you would get away without implementing your own little Logistic Regression? Hah!


# Exercise 1. Implement the Sigmoid Function
*aka the logistic function*

As a very simple warmup, you will implement the sigmoid function. Let's keep this simple!

Here's a quick reminder of the formula:

$$\hat{p} = \frac{1}{1 + e^{-z}}$$

**Complete here:**

In [2]:
def sigmoid_function(z):
    """ 
    Implementation of the sigmoid function by hand
    
    Args:
        z (np.float64): a float

    Returns:
        proba (np.float64): the predicted probability for a given observation

    """
    
    # obtain the predicted probability 
    # clue: you can use np.exp()
    proba = 1/(1+np.exp(-z))
    # YOUR CODE HERE
    return proba

In [3]:
z = 0.2
print('Predicted probability: %.2f' % sigmoid_function(z))

Predicted probability: 0.55


Expected output:

    Predicted probability: 0.55

In [4]:
z = 1.1
assert hashlib.md5(np.round(sigmoid_function(z),2)).hexdigest() == 'c12ba117c0d1805163e06e53c8c3bd81'

z = -5
assert hashlib.md5(np.round(sigmoid_function(z),2)).hexdigest() == '815d609ae70b810f80cec52c8d9349bc'

# Exercise 2: Make Predictions From Observations

The next step is to implement a function that receives observations and returns predicted probabilities.

For instance, remember that for an observation with two variables we have:

$$z = \beta_0 + \beta_1 x_1 + \beta_2 x_2$$

where $\beta_0$ is the intercept and $\beta_1, \beta_2$ are the coefficients.

**Complete here:**

In [5]:
def predict_proba(x_train, coefficients):
    """ 
    Implementation of a function that returns predicted probabilities for a given set of data samples
    
    Args:
        x_train (np.array): a numpy array of shape (m,n)
            - m: number of training observations
            - n: number of variables
        coefficients (np.array): a numpy array of shape (n + 1,)
            - coefficients[0]: intercept
            - coefficients[1:]: remaining coefficients

    Returns:
        proba (np.array): the predicted probabilities for a given set of data samples

    """

    # start by assigning the intercept to z 
    # clue: the intercept is the first element of the list of coefficients
    z = coefficients[0]
    # YOUR CODE HERE
    z+=np.dot(x_train,coefficients[1:])
    
    # sum the remaining variable * coefficient products to z
    # clue: the variables and coefficients indeces are not exactly aligned, but correctly ordered
    #for i in range():                     # replace the empty list with range and iterate through the observation variables (clue: you can use np.shape())
    #    z += None                    # multiply the variable value by its coefficient and add to z (clue: use x[:, 0] to access the first variable over all samples)
    # YOUR CODE HERE
    #raise NotImplementedError()
    
    # obtain the predicted probability from z
    # clue: we already implemented something that can give us that
    proba = sigmoid_function(z)
    # YOUR CODE HERE
    #raise NotImplementedError()

    return proba

In [6]:
x = np.array([[1.3, 1.1, 0.4, 2.2],
             [1.0, 2.0, 0.5, 0.1]])
coefficients = np.array([1.1,5,-4, 10, -3])

print('Predicted probabilities:  %.3f and %.3f' % (predict_proba(x, coefficients)[0], predict_proba(x, coefficients)[1]))

Predicted probabilities:  0.646 and 0.943


Expected output:

    Predicted probabilities:  0.646 and 0.943

In [7]:
x = np.array([[2, 0.1, -1.1], [-1.2, 1.2, 3.2]])
coefficients = np.array([-0.1,1,-3, 1.3])
assert hashlib.md5(np.round(predict_proba(x, coefficients),2)).hexdigest() == 'e1e3a8f137e127c9c8f047843ba9a187'

# Exercise 3: Compute the Maximum Log-Likelihood Cost Function

As you will implement stochastic gradient descent, you only have to do the following for each prediction: 

$$H_{\hat{p}}(y) =  - (y \log(\hat{p}) + (1-y) \log (1-\hat{p}))$$

**Complete here:**

In [8]:
def maximum_log_likelihood(y, proba):
    """ 
    Implementation of a function that returns the Cross-Entropy loss
    
    Args:
        y (np.int64): an integer
        proba (np.float64): a float

    Returns:
        loss (np.float): a float with the resulting loss for a given prediction

    """
    
    # compute the inner right side of the loss function (for when y == 0)
    inner_right = (1-y)*np.log(1-proba)
    # YOUR CODE HERE
    
    
    # compute the inner left side of the loss function (for when y == 1)
    # clue: use np.log()
    inner_left = y*np.log(proba)
    # YOUR CODE HERE
    
    # compute the total loss
    total_loss = -(inner_right + inner_left)
    # YOUR CODE HERE
    return total_loss

In [9]:
y = 1
proba = 0.2
print('Computed loss:  %.3f' % maximum_log_likelihood(y, proba))

Computed loss:  1.609


Expected output:
    
    Computed loss:  1.609

In [10]:
y = 1
proba = 0.11
assert hashlib.md5(np.round(maximum_log_likelihood(y, proba),3)).hexdigest() == 'c32ab5f1bb15b402788647e939fca19c'

y = 1
proba = 0.98
assert hashlib.md5(np.round(maximum_log_likelihood(y, proba),3)).hexdigest() == '3c0098685eae68ef1f7ce8a5bbd1c641'

# Exercise 4: Obtain the Optimized Coefficients 

Now that the warmup is done, let's do the most interesting exercise. Here you will implement the optimized coefficients through Batch Gradient Descent.

## Quick reminders:

For Stochastic gradient descent:

$$H_{\hat{p}}(y) = - \frac{1}{N}\sum_{i=1}^{N} \left [{ y_i \ \log(\hat{p}_i) + (1-y_i) \ \log (1-\hat{p}_i)} \right ]$$

and

$$\beta_{0(t+1)} = \beta_{0(t)} - learning\_rate \frac{\partial H_{\hat{p}}(y)}{\partial \beta_{0(t)}}$$

$$\beta_{t+1} = \beta_t - learning\_rate \frac{\partial H_{\hat{p}}(y)}{\partial \beta_t}$$

which can be simplified to

$$\beta_{0(t+1)} = \beta_{0(t)} + learning\_rate \left [(y - \hat{p}) \ \hat{p} \ (1 - \hat{p})\right]$$

$$\beta_{t+1} = \beta_t + learning\_rate \left [(y - \hat{p}) \ \hat{p} \ (1 - \hat{p}) \ x \right]$$

Then for Batch Gradient Descent:

$$\beta_{t+1} = \beta_t + learning\_rate \frac{1}{N} \sum_{i=1}^{N} \left [(y_i - \hat{p}_i) \ \hat{p}_i \ (1 - \hat{p}_i) \ x_i \right]$$

And the intercept:

$$\beta_{0(t+1)} = \beta_{0(t)} + learning\_rate \frac{1}{N} \sum_{i=1}^{N} \left [(y_i - \hat{p}_i) \ \hat{p}_i \ (1 - \hat{p}_i) \right]$$

You will have to initialize a numpy array full of zeros for the coefficients. If you have a training set $X$, you can initialize it this way:
```python
coefficients = np.zeros(X.shape[1]+1)
```

where the $+1$ is adding the intercept.

**Complete here:**

In [11]:
def compute_coefficients(x_train, y_train, learning_rate = 0.1, n_epoch = 50, verbose = False):
    """ 
    Implementation of a function that returns the optimized intercept and coefficients

    Args:
        x_train (np.array): a numpy array of shape (m, n)
            m: number of training observations
            n: number of variables
        y_train (np.array): a numpy array of shape (m,)
        learning_rate (np.float64): a float
        n_epoch (np.int64): an integer of the number of full training cycles to perform on the training set

    Returns:
        coefficients (np.array): a numpy array of shape (n+1,)

    """

    # initialize the coefficients array with zeros
    # clue: use np.zeros()
    coefficients = np.zeros(x_train.shape[1]+1)
    # YOUR CODE HERE
    

    # run the batch gradient descent algorithm n_epoch times and update the coefficients
    for epoch in range(n_epoch):                    # replace the empty list with range and iterate n_epoch times
        proba = predict_proba(x_train,coefficients)
        # compute the predicted probability
        #print('proba: '+ str(proba))
        #print('coefs_pre'+ str(coefficients))
        coefficients[0] += learning_rate*np.mean((y_train-proba)*proba*(1-proba))
        #print((y_train-proba)*proba*(1-proba))
        #print('coefs_pos'+ str(coefficients))# update the intercept (clue: use np.mean())      
        #print('update_value'+ str((y_train-proba)*proba*(1-proba)))
        for i in range(len(coefficients)-1):                    # replace the empty list with range and iterate through the data variables (clue: use np.shape())
            #print('yo'+str(np.dot((y_train-proba)*proba*(1-proba),x_train)))
            #print(x_train[:,i])
            coefficients[i + 1] += learning_rate*np.mean(((y_train-proba)*proba*(1-proba)) * x_train[:,i]) # update each coefficient (clue: use np.mean())
        loss =   np.mean(maximum_log_likelihood(y_train, proba))
        #print('coefs_pos'+ str(coefficients))
        #print(maximum_log_likelihood(y_train,  predict_proba(x_train,coefficients)))# average the obtained cross entropies (clue: use np.mean())
        # YOUR CODE HERE

        if((epoch%10==0) & verbose):
            print('>epoch=%d, learning_rate=%.3f, error=%.3f' % (epoch, learning_rate, loss))
    return coefficients

In [12]:
x_train = np.array([[1,2,3], [2,5,9], [3,1,4], [8,2,9]])
y_train = np.array([0,1,0,1])
learning_rate = 0.1
n_epoch = 200
coefficients = compute_coefficients(x_train, y_train, learning_rate=learning_rate, n_epoch=n_epoch, verbose=True)
print('Computed coefficients:')
print(coefficients)

>epoch=0, learning_rate=0.100, error=0.693
>epoch=10, learning_rate=0.100, error=0.607
>epoch=20, learning_rate=0.100, error=0.599
>epoch=30, learning_rate=0.100, error=0.592
>epoch=40, learning_rate=0.100, error=0.584
>epoch=50, learning_rate=0.100, error=0.576
>epoch=60, learning_rate=0.100, error=0.569
>epoch=70, learning_rate=0.100, error=0.562
>epoch=80, learning_rate=0.100, error=0.555
>epoch=90, learning_rate=0.100, error=0.548
>epoch=100, learning_rate=0.100, error=0.541
>epoch=110, learning_rate=0.100, error=0.534
>epoch=120, learning_rate=0.100, error=0.527
>epoch=130, learning_rate=0.100, error=0.521
>epoch=140, learning_rate=0.100, error=0.514
>epoch=150, learning_rate=0.100, error=0.508
>epoch=160, learning_rate=0.100, error=0.502
>epoch=170, learning_rate=0.100, error=0.496
>epoch=180, learning_rate=0.100, error=0.490
>epoch=190, learning_rate=0.100, error=0.484
Computed coefficients:
[-0.77946284 -0.02035626 -0.04034926  0.23211248]


Expected output:

    >epoch=0, learning_rate=0.100, error=0.693
    >epoch=10, learning_rate=0.100, error=0.607
    >epoch=20, learning_rate=0.100, error=0.599
    >epoch=30, learning_rate=0.100, error=0.592
    >epoch=40, learning_rate=0.100, error=0.584
    >epoch=50, learning_rate=0.100, error=0.576
    >epoch=60, learning_rate=0.100, error=0.569
    >epoch=70, learning_rate=0.100, error=0.562
    >epoch=80, learning_rate=0.100, error=0.555
    >epoch=90, learning_rate=0.100, error=0.548
    >epoch=100, learning_rate=0.100, error=0.541
    >epoch=110, learning_rate=0.100, error=0.534
    >epoch=120, learning_rate=0.100, error=0.527
    >epoch=130, learning_rate=0.100, error=0.521
    >epoch=140, learning_rate=0.100, error=0.514
    >epoch=150, learning_rate=0.100, error=0.508
    >epoch=160, learning_rate=0.100, error=0.502
    >epoch=170, learning_rate=0.100, error=0.496
    >epoch=180, learning_rate=0.100, error=0.490
    >epoch=190, learning_rate=0.100, error=0.484
    Computed coefficients:
    [-0.77946284 -0.02035626 -0.04034926  0.23211248]

In [13]:
x_train = np.array([[3,1,3], [1,0,9], [3,3,4], [2,-1,10]])
y_train = np.array([0,1,0,1])
learning_rate = 0.3
n_epoch = 200
coefficients = compute_coefficients(x_train, y_train, learning_rate=learning_rate, n_epoch=n_epoch, verbose=False)
assert np.allclose(coefficients, np.array([-0.22368326, -0.98713492, -0.7708793 ,  0.4834609 ]))

x_train = np.array([[3,-1,-2], [-6,9,3], [3,-1,4], [5,1,6]])
coefficients = compute_coefficients(x_train, y_train, learning_rate=learning_rate, n_epoch=n_epoch, verbose=False)
assert np.allclose(coefficients, np.array([-0.40768566, -0.21773974,  1.65415715,  0.28374746]))

# Exercise 5: Normalize Data

Just a quick and easy function to normalize the data. It is crucial that your variables are adjusted between $[0;1]$ (normalized) or standardized so that you can correctly analyse some logistic regression coefficients for your possible future employer.

You only have to implement this formula

$$ x_{normalized} = \frac{x - x_{min}}{x_{max} - x_{min}}$$

Don't forget that the `axis` argument is critical when obtaining the maximum, minimum and mean values! As you want to obtain the maximum and minimum values of each individual feature, you have to specify `axis=0`. Thus, if you wanted to obtain the maximum values of each feature of data $X$, you would do the following:

```python
X_max = np.max(X, axis=0)
```

**Complete here:**

In [14]:
def normalize_data(X):
    """ 
    Implementation of a function that normalizes your data variables
    
    Args:
        X (np.array): a numpy array of shape (m, n)
            m: number of observations
            n: number of variables

    Returns:
        normalized_data (np.array): a numpy array of shape (m, n)

    """
    # compute the numerator
    # clue: use np.min()
    numerator = X-np.min(X,axis=0)
    # YOUR CODE HERE
    
    # compute the numerator
    # clue: use np.max() and np.min()
    denominator = np.max(X,axis=0)-np.min(X,axis=0)
    # YOUR CODE HERE
    
    # obtain the normalized data
    normalized_X = numerator/denominator
    # YOUR CODE HERE
    
    return normalized_X

In [15]:
data = np.array([[7,7,3], [2,2,11], [9,5,2], [0,9,5], [10,1,3], [1,5,2]])
normalized_data = normalize_data(data)
print('Before normalization:')
print(data)
print('\n-------------------\n')
print('After normalization:')
print(normalized_data)

Before normalization:
[[ 7  7  3]
 [ 2  2 11]
 [ 9  5  2]
 [ 0  9  5]
 [10  1  3]
 [ 1  5  2]]

-------------------

After normalization:
[[0.7        0.75       0.11111111]
 [0.2        0.125      1.        ]
 [0.9        0.5        0.        ]
 [0.         1.         0.33333333]
 [1.         0.         0.11111111]
 [0.1        0.5        0.        ]]


Expected output:
    
    Before normalization:
    [[ 7  7  3]
     [ 2  2 11]
     [ 9  5  2]
     [ 0  9  5]
     [10  1  3]
     [ 1  5  2]]

    -------------------

After normalization:

    [[0.7        0.75       0.11111111]
     [0.2        0.125      1.        ]
     [0.9        0.5        0.        ]
     [0.         1.         0.33333333]
     [1.         0.         0.11111111]
     [0.1        0.5        0.        ]]

In [16]:
data = np.array([[2,2,11,1], [7,5,1,3], [9,5,2,6]])
normalized_data = normalize_data(data)
assert hashlib.md5(normalized_data).hexdigest() == '277af6f0e6721a66ca19931c726aeb86'

data = np.array([[1,3,1,3], [9,5,3,1], [2,2,4,6]])
normalized_data = normalize_data(data)
assert hashlib.md5(normalized_data).hexdigest() == '2548d399591b9f950ab5fed9cc89a4e5'

# Exercise 6: Putting it All Together (Optional)

Do this exercise if you want to apply it to a real classification problem.

## The Banknote Authentication Dataset

There are 1372 items (images of banknotes — think Euro or dollar bill). There are 4 predictor variables (variance of image, skewness, kurtosis, entropy). The variable to predict is encoded as 0 (authentic) or 1 (forgery).

Your quest, is to first analyze this dataset from the materials that you've learned in the previous SLUs and then create a logistic regression model that can correctly classify forged banknotes from authentic ones.

The data is loaded for you below.

In [17]:
columns = ['variance','skewness','kurtosis','entropy', 'forgery']
data = pd.read_csv('data/data_banknote_authentication.txt',names=columns).sample(frac=1, random_state=1)
X = data.drop(columns='forgery').values
y_train = data.forgery.values

In [18]:
data

,variance,skewness,kurtosis,entropy,forgery
1240,-3.551000,1.895500,0.186500,-2.440900,1
703,1.311400,4.546200,2.293500,0.225410,0
821,-4.017300,-8.312300,12.454700,-1.437500,1
1081,-5.119000,6.648600,-0.049987,-6.520600,1
37,3.628900,0.813220,1.627700,0.776270,0
167,1.023500,6.901000,-2.006200,-2.712500,0
223,4.646400,10.532600,-4.585200,-4.206000,0
647,3.779100,2.576200,1.309800,0.565500,0
325,4.454900,2.497600,1.031300,0.968940,0
558,4.384600,-4.879400,3.366200,-0.029324,0


You will also have to return several values, such as the number of forged and authentic banknotes. To do so, remember that you can do masks in numpy arrays. If you had a numpy array of labels called `labels` and wanted to obtain the ones with label $1$, you would do the following:

```python
filtered_labels = labels[labels==1]
```

You will additionally be asked to obtain the number of correct forged banknotes predictions. Imagine that you have a numpy array with the predictions called `predictions` and a numpy array with the correct labels called `labels` and you wanted to obtain the number of correct predictions of a label $1$. You would do the following:

```python
n_correct_predictions = labels[(labels==1) & (predictions==1)].shape[0]
```

Also, don't forget to use these values for your logistic regression!

In [19]:
# Hyperparameters
learning_rate = 0.3
n_epoch = 200

# For validation
verbose = True

Now let's do this!

**Complete here:**

In [20]:
len(y_train)

1372

In [21]:
from sklearn.linear_model import LogisticRegression
# STEP ONE: Initial analysis and data processing
# How many banknotes are forged? (clue: use y_train)
n_forged = y_train.sum()
# YOUR CODE HERE
# How many banknotes are authentic? (clue: use y_train)
n_authentic = len(y_train) - n_forged
# YOUR CODE HERE

# Normalize the training data X (clue: we have already implemented this)
x_train = normalize_data(X)
# YOUR CODE HERE

print("Number of forged banknotes: %i" % n_forged)

print("\nThe last three normalized rows:")
print(x_train[-3:])


Number of forged banknotes: 610

The last three normalized rows:
[[0.19293425 0.74247045 0.25236091 0.28018586]
 [0.65542407 0.59132937 0.3214595  0.76966693]
 [0.34091253 0.65257608 0.19769531 0.6638479 ]]



Expected output:

    Number of forged banknotes: 610

    The last three normalized rows:
    [[0.19293425 0.74247045 0.25236091 0.28018586]
     [0.65542407 0.59132937 0.3214595  0.76966693]
     [0.34091253 0.65257608 0.19769531 0.6638479 ]]

In [22]:
# STEP TWO: Model training and predictions
# What coefficients can we get? (clue: we have already implemented this)
# note: don't forget to use all the hyperparameters defined above
coefficients = compute_coefficients(x_train, y_train, learning_rate = 0.1, n_epoch = 50, verbose = False)
# YOUR CODE HERE

# What are the predicted probabilities on the training data?
probas = predict_proba(x_train,coefficients)   # get the predicted probabilities (clue: we already implemented this)
    
# YOUR CODE HERE

# If we had to say whether a banknote was forged or not, what are the predictions?
# clue 1: Hard assign the predicted probabilities by rounding them to the nearest integer
# clue 2: use np.round()
preds = np.round(probas)
# YOUR CODE HERE

print("\nThe last three coefficients:")
print(coefficients[-3:])

print("\nThe last three obtained probas:")
print(probas[-3:])

print("\nThe last three predictions:")
print(preds[-3:])


The last three coefficients:
[-0.08321107  0.00576783 -0.02943804]

The last three obtained probas:
[0.46747167 0.45417692 0.46229941]

The last three predictions:
[0. 0. 0.]


Expected output:

    >epoch=0, learning_rate=0.300, error=0.693
    >epoch=10, learning_rate=0.300, error=0.680
    >epoch=20, learning_rate=0.300, error=0.673
    >epoch=30, learning_rate=0.300, error=0.667
    >epoch=40, learning_rate=0.300, error=0.662
    >epoch=50, learning_rate=0.300, error=0.658
    >epoch=60, learning_rate=0.300, error=0.654
    >epoch=70, learning_rate=0.300, error=0.650
    >epoch=80, learning_rate=0.300, error=0.647
    >epoch=90, learning_rate=0.300, error=0.643
    >epoch=100, learning_rate=0.300, error=0.639
    >epoch=110, learning_rate=0.300, error=0.636
    >epoch=120, learning_rate=0.300, error=0.632
    >epoch=130, learning_rate=0.300, error=0.629
    >epoch=140, learning_rate=0.300, error=0.626
    >epoch=150, learning_rate=0.300, error=0.623
    >epoch=160, learning_rate=0.300, error=0.620
    >epoch=170, learning_rate=0.300, error=0.617
    >epoch=180, learning_rate=0.300, error=0.614
    >epoch=190, learning_rate=0.300, error=0.611

    The last three coefficients:
    [-0.46533509  0.21801537  0.13774932]

    The last three obtained probas:
    [0.46480018 0.40605379 0.45419773]

    The last three predictions:
    [0. 0. 0.]

In [23]:
preds[preds==1]

array([], dtype=float64)

In [24]:
# STEP THREE: Results analysis
# How many banknotes were predicted as forged? (clue: use preds and len() or .shape)
n_predicted_forged = (preds==1).sum()
# YOUR CODE HERE

# How many forged banknotes were correctly detected? (clue: use y_train, preds and len() or .shape)
n_correct_forged_predictions = ((y_train==1) & (preds==1)).sum()
# YOUR CODE HERE

print("Number of correct forged predictions: %i" % n_correct_forged_predictions)

Number of correct forged predictions: 0


Expected output:

    Number of correct forged predictions: 165

In [25]:
print('You have a dataset with %s forged banknotes and %s authentic banknotes. \n\n'
     'After analysing the data and training your own logistic regression classifier you find out that it correctly '
     'identified %s out of %s forged banknotes. But you know that it still did not detect most of the forged banknotes.'% 
      (n_forged, n_authentic, n_predicted_forged, n_correct_forged_predictions))

You have a dataset with 610 forged banknotes and 762 authentic banknotes. 

After analysing the data and training your own logistic regression classifier you find out that it correctly identified 0 out of 0 forged banknotes. But you know that it still did not detect most of the forged banknotes.
